In [1]:
 import tensorflow as tf # Imports tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Embedding,TimeDistributed
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ReLU, LSTM,Bidirectional,Attention,Concatenate,concatenate
from tensorflow.keras import regularizers, optimizers,losses
from tensorflow.keras.layers import DepthwiseConv2D,Add, ReLU, GlobalAveragePooling2D, GlobalMaxPooling2D,MultiHeadAttention
from tensorflow.keras.layers import Activation,ActivityRegularization, AvgPool2D, LeakyReLU, Conv2DTranspose
from tensorflow.keras.metrics import Accuracy,Recall,Precision,AUC,TruePositives,TrueNegatives,FalseNegatives,FalsePositives, SpecificityAtSensitivity,SensitivityAtSpecificity
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import sklearn
#import imblearn
#from keras.optimizers import Adam
import matplotlib.pyplot as plt
import time
import os
import sklearn.metrics as m
from glob import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import skimage.io
import skimage.color
import skimage.filters

In [3]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/
%ls

Mounted at /content/drive
/content/drive/MyDrive
 0/
 1/
 20102020-CFP-V3.docx
 338.mp4
 47.gdoc
'6th Jan'
'A4 template doc.gdoc'
'Abstract for HCII 2023_MMR.docx'
 Abstract.gdoc
 A_Human_Robot_Interaction_System_highlighted.pdf
 all.jpg
"Alzheimer's 21-22.csv"
 anaconda.gdoc
'Anik CV.gdoc'
'Application .gdoc'
'Audio Test.gdoc'
'Author Agreement.gdoc'
'Author Response .gdoc'
'Autism update 29.gdoc'
'Automation aided farming.gdoc'
'Ayatul kudsi.gdoc'
 ba.pdf
'basic code.gdoc'
'Bio Graphy.gdoc'
'Biography_Yousuf 1.gdoc'
'Blockchain Interoperability (swap).gdoc'
 Bot.gdoc
'BT paper Review.gdoc'
 car-segment_maskseg.png
 cel0.jpg
 cel1.jpg
 cel2.jpg
 cel3.jpg
 Certficates/
 Classroom/
 CNN.png
'Colab Notebooks'/
 Contribution_el.docx
 Conversation.gdoc
'Copy of 3-2 report.docx'
'Copy of FASTEN Related Works.gsheet'
'Count Feas.csv'
'Count Feas.gsheet'
'Cover Letter Elsevier.gdoc'
'cover letter sabbir.gdoc'
'CP Community'/
'Credit Author Statement.gdoc'
 Cross_val_UNET.gsheet
'cyber securit

In [11]:
images=[]
masks=[]

y=[]
gr_shape=(128,128,1)
img_shape = (128,128,3)
Thres = 1000
input_dir="/content/drive/MyDrive/Research/Lung Cancer/Datasets/LIDC-IDRI-slices/"
out_dir = "/content/drive/MyDrive/Research/Lung Cancer/ModelRun"


In [12]:
for patient in os.listdir(input_dir):
    for nodule in os.listdir(input_dir+'/'+patient):
        mask0=[]
        mask1=[]
        mask2=[]
        mask3=[]
        temp_image=[]
        for filename in glob(input_dir+'/'+patient+'/'+nodule+'/images/*.png'):
            img = image.load_img(filename,target_size=img_shape)
            img = np.asarray(img)
            temp_image.append(img)
        for filename in glob(input_dir+'/'+patient+'/'+nodule+'/mask-0/*.png'):
            img = image.load_img(filename,target_size=img_shape)
            img = np.asarray(img)
            mask0.append(img)
        for filename in glob(input_dir+'/'+patient+'/'+nodule+'/mask-1/*.png'):
            img = image.load_img(filename,target_size=img_shape)
            img = np.asarray(img)
            mask1.append(img)
        for filename in glob(input_dir+'/'+patient+'/'+nodule+'/mask-2/*.png'):
            img = image.load_img(filename,target_size=img_shape)
            img = np.asarray(img)
            mask2.append(img)
        for filename in glob(input_dir+'/'+patient+'/'+nodule+'/mask-3/*.png'):
            img = image.load_img(filename,target_size=img_shape)
            img = np.asarray(img)
            mask3.append(img)
        for i in range(len(mask0)):
            white_sum=np.array([mask0[i].sum(),mask1[i].sum(),mask2[i].sum(),mask3[i].sum()])
            cnt=int(mask0[i].sum()>Thres)+int(mask1[i].sum()>Thres)+int(mask2[i].sum()>Thres)+int(mask3[i].sum()>Thres)
            images.append(temp_image[i])
            if(cnt>2):
                y.append(1)
                if white_sum.argmax()==0:
                    masks.append(mask0[i])
                elif white_sum.argmax()==1:
                    masks.append(mask1[i])
                elif white_sum.argmax()==2:
                    masks.append(mask2[i])
                else:
                    masks.append(mask3[i])
            else:
                y.append(0)
                if white_sum.argmin()==0:
                    masks.append(mask0[i])
                elif white_sum.argmin()==1:
                    masks.append(mask1[i])
                elif white_sum.argmin()==2:
                    masks.append(mask2[i])
                else:
                    masks.append(mask3[i])

KeyboardInterrupt: 

In [13]:
images=np.array(images)
masks=np.array(masks)
label=np.array(y)
print(images.shape)
print(masks.shape)
print(label.shape)


(12214, 128, 128, 3)
(12214, 128, 128, 3)
(12214,)


In [14]:
yy=pd.DataFrame(label)
yy.value_counts()
x_train,x_test,y_train,y_test,mask_train,mask_test = train_test_split(images,label,masks,train_size=0.8, random_state=7)
print(x_train.shape)
print(y_train.shape)

(9771, 128, 128, 3)
(9771,)


In [17]:
from keras.models import load_model
model = tf.keras.models.load_model('/content/drive/MyDrive/Research/Lung Cancer/ModelRun/Unet_LIDC/lidcUnet0.h5')

In [18]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import adapted_rand_error
from skimage.metrics import hausdorff_distance
from skimage.metrics import mean_squared_error
from skimage.metrics import normalized_root_mse
from skimage.metrics import normalized_mutual_information
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import variation_of_information


In [ ]:
smooth=1.0
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return dice_coef_loss(y_true, y_pred) + bce(y_true, y_pred)


def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac



In [19]:
pred_mask_test = model.predict(x_test)

77/77 [==============================] - 239s 3s/step


In [ ]:
v_PSNR = PSNR(mask_test,pred_mask_test)
v_bce_dl = bce_dice_loss(mask_test,pred_mask_test)
v_dc = dice_coef(mask_test,pred_mask_test)
v_dl = dice_coef_loss(mask_test,pred_mask_test)
v_iou = iou(mask_test,pred_mask_test)

In [ ]:
result= [['PSNR', 'BCE-DL', 'IOU', 'DC', 'DL']]
result.append([v_PSNR ,v_bce_dl, v_iou ,v_dc ,v_dl])
print(pd.DataFrame(result))

       0       1      2      3      4
0   PSNR  BCE-DL    IOU     DC     DL
1  0.893   0.014  0.734  0.967  0.033


In [20]:
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import adapted_rand_error as ARE
from skimage.metrics import hausdorff_distance as HD
from skimage.metrics import mean_squared_error as MSE
from skimage.metrics import normalized_root_mse as Nrmse
from skimage.metrics import normalized_mutual_information as NMI
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import variation_of_information as VOI
smooth=1.0
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):

    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    return dice_coef_loss(y_true, y_pred) + bce(y_true, y_pred)


def iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    sum_ = np.sum(y_true + y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac

result= [['PSNR', 'IOU', 'DC', 'MSE','NRMSE','NMI']]

v_PSNR = PSNR(mask_test,pred_mask_test)
#v_bce_dl = bce_dice_loss(mask_test2,pred_mask_test)
v_dc = dice_coef(mask_test,pred_mask_test)
v_dl = dice_coef_loss(mask_test,pred_mask_test)
v_iou = iou(mask_test,pred_mask_test)
#v_ssim = SSIM(mask_test2,pred_mask_test,data_range=1, channel_axis=3)
#v_are = ARE(mask_test,pred_mask_test)
#v_hd = HD(mask_test,pred_mask_test)
v_mse = MSE(mask_test,pred_mask_test)
v_nrmse = Nrmse(mask_test,pred_mask_test)
v_nmi = NMI(mask_test,pred_mask_test)
#v_voi = VOI(mask_test,pred_mask_test)
result.append([v_PSNR , v_iou ,v_dc ,v_mse,v_nrmse,v_nmi])

<ipython-input-20-28d88a5b3cf3>:36: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  v_PSNR = PSNR(mask_test,pred_mask_test)


In [32]:
x_train,x_test,y_train,y_test,mask_train,mask_test = train_test_split(images,label,masks,train_size=0.85, random_state=17)

In [33]:
pred_mask_test = model.predict(x_test)

58/58 [==============================] - 179s 3s/step


In [34]:
v_PSNR = PSNR(mask_test,pred_mask_test)
#v_bce_dl = bce_dice_loss(mask_test2,pred_mask_test)
v_dc = dice_coef(mask_test,pred_mask_test)
v_dl = dice_coef_loss(mask_test,pred_mask_test)
v_iou = iou(mask_test,pred_mask_test)
#v_ssim = SSIM(mask_test2,pred_mask_test,data_range=1, channel_axis=3)
#v_are = ARE(mask_test,pred_mask_test)
#v_hd = HD(mask_test,pred_mask_test)
v_mse = MSE(mask_test,pred_mask_test)
v_nrmse = Nrmse(mask_test,pred_mask_test)
v_nmi = NMI(mask_test,pred_mask_test)
#v_voi = VOI(mask_test,pred_mask_test)
result.append([v_PSNR , v_iou ,v_dc ,v_mse,v_nrmse,v_nmi])

<ipython-input-34-f9f0ca70b706>:1: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  v_PSNR = PSNR(mask_test,pred_mask_test)


In [35]:
print(pd.DataFrame(result))

           0         1         2           3         4         5
0       PSNR       IOU        DC         MSE     NRMSE       NMI
1  22.045977   1.60031  1.230861  405.959436    0.9973  1.000032
2  22.045977   1.60031  1.230861  405.959436    0.9973  1.000032
3   21.95924  1.632739  1.240335  414.148762  0.997276  1.000024
4  21.862151  1.669846  1.250894  423.511581  0.997249  1.000011
